# GAN Training

## imports

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from models.GAN import GAN
from utils.loaders import load_safari

from PIL import Image
from numpy import asarray


In [ ]:
directory = 'data/datasetdrawing'
count = 0
for filename in os.listdir(directory):
    count+=1
    
dataset = np.zeros([count,28*28])
count = 0

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    # load the image
    im = np.array(Image.open(f).convert('L')) #you can pass multiple arguments in single line
    # create Pillow image
    image2 = Image.fromarray(im)
    
    # Resize image and convert in greycolor
    load_img_rz = np.array(Image.fromarray(im).resize((28,28)))
    # Save the image in jpeg
    Image.fromarray(load_img_rz).save('r_'+filename)
    
    image2 = Image.fromarray(load_img_rz)
    
    x = np.zeros([1,load_img_rz.size])
    k = 0
    for i in range(28):
        for j in range(28):
            x[0,k] = load_img_rz[i,j]
            k+=1
    dataset[count,:] = x[0,:]
    count +=1        


np.save('./data/dataset.npy', dataset)

In [ ]:
# run params
SECTION = 'gan'
RUN_ID = '0001'
DATA_NAME = 'drawing'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


## data

In [ ]:
(x_train, y_train) = load_safari(DATA_NAME)

In [ ]:
x_train.shape

In [ ]:
plt.imshow(x_train[200,:,:,0], cmap = 'gray')

## architecture

In [ ]:
gan = GAN(input_dim = (28,28,1)
        , discriminator_conv_filters = [64,64,128,128]
        , discriminator_conv_kernel_size = [5,5,5,5]
        , discriminator_conv_strides = [2,2,2,1]
        , discriminator_batch_norm_momentum = None
        , discriminator_activation = 'relu'
        , discriminator_dropout_rate = 0.4
        , discriminator_learning_rate = 0.0008
        , generator_initial_dense_layer_size = (7, 7, 64)
        , generator_upsample = [2,2, 1, 1]
        , generator_conv_filters = [128,64, 64,1]
        , generator_conv_kernel_size = [5,5,5,5]
        , generator_conv_strides = [1,1, 1, 1]
        , generator_batch_norm_momentum = 0.9
        , generator_activation = 'relu'
        , generator_dropout_rate = None
        , generator_learning_rate = 0.0004
        , optimiser = 'rmsprop'
        , z_dim = 100
        )

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [ ]:
gan.discriminator.summary()

In [ ]:
gan.generator.summary()

## training

In [ ]:
BATCH_SIZE = 90
EPOCHS = 6000
PRINT_EVERY_N_BATCHES = 5

In [ ]:
gan.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
)

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, 2000)
plt.ylim(0, 2)

plt.show()


In [ ]:
fig = plt.figure()
plt.plot([x[3] for x in gan.d_losses], color='black', linewidth=0.25)
plt.plot([x[4] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[5] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot([x[1] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('accuracy', fontsize=16)

plt.xlim(0, 2000)

plt.show()